In [2]:
import numpy as np
import matplotlib.pyplot as plt


# **Antithetic variables**


**Monte Carlo Estimator:**

The estimator for the expectation 
$$
I = \mathbb{E}[f(X)] = \int_{x} f(x) p(x) \,dx
$$ 
using  N  i.i.d. samples is given by:
$$
\hat{I}_{MC} = \frac{1}{N} \sum_{i=1}^{N} f(X_i)
$$
with variance
$$
\text{Var}(\hat{I}_{MC}) = \frac{\text{Var}(f(X))}{N}
$$


**Antithetic Variate Estimator:**

Using antithetic variates, we form pairs \((X_i, X'_i)\) and define the estimator as:

$$
\hat{I}_{AV} = \frac{1}{2N} \sum_{i=1}^{N} \left( f(X_i) + f(X'_i) \right)=\hat{I}_{MC}.
$$

Its variance is given by:

$$
\text{Var}(\hat{I}_{AV}) = \frac{1}{4N}\text{Var}\Bigl( f(X) + f(X') \Bigr) = \frac{1}{2N} \left[ \text{Var}(f(X)) + \text{Cov}(f(X), f(X')) \right]< \frac{1}{2N} \text{Var}(f(X)) < \frac{1}{N} \text{Var}(f(X))=\text{Var}(\hat{I}_{MC}) .
$$

If **covariance** is negative so that $X$ and $X'$ are negatively correlated. We can use inversion to create $X'$ from $X$


## **Example 1:**

$$I = E\left[e^X\right]$$


$$
X_1, X_2, \dots, X_n \sim \text{Uniform}(0,1) \quad \text{i.i.d.}
$$

In [12]:
def f(x):
    return np.exp(x)
N=10**3
X = np.random.uniform(0, 1, N)
#inversion and set increasing on x or decreasing on x 
X_prime=1-X


print("#expectation estimation ",np.mean(f(X)))
print("#variance naive ",np.var(f(X))/N)
print("#variance AV ",np.var(f(X)+f(X_prime))/(4*N))
#  

#expectation estimation  1.734351270781523
#variance naive  0.00024256798799387753
#variance AV  3.900990751005053e-06


## **Example 2:**



$$I = E\left[e^X\right]$$


$$
X_1, X_2, \dots, X_n \sim \text{N}(0,1) \quad \text{i.i.d.}
$$

In [53]:
def f(x):
    return np.exp((x))
N=10**3
U = np.random.uniform(0, 1, 10000)
V = np.random.uniform(0, 1, 10000)

# Apply Box-Muller Transform
X = np.sqrt(-2 * np.log(U)) * np.cos(2 * np.pi * V)
X_prime=-X


print("#expectation estimation ",np.mean(f(X)))
print("#variance naive ",np.var(f(X))/N)
print("#variance AV ",np.var(f(X)+f(X_prime))/(4*N))
#  

#expectation estimation  1.627964370774247
#variance naive  0.0043145590008988235
#variance AV  0.0014024509613161735


## **Example 3:**



$$I = E\left[e^X\right]$$


$$
X_1, X_2, \dots, X_n \sim \text{Exp}(\lambda) \quad \text{i.i.d.}
$$

In [64]:
def f(x):
    return np.exp((x))
N=10**3
lambda_val = 2  # Rate parameter for Exponential(λ)
# Generate uniform random numbers
U = np.random.uniform(0, 1, 10000)
# Apply inverse transformation
X = -np.log( U) / lambda_val
X_prime = -np.log(1- U) / lambda_val

print("#expectation estimation ",np.mean(f(X)))
print("#variance naive ",np.var(f(X))/N)
print("#variance AV ",np.var(f(X)+f(X_prime))/(4*N))
 

#expectation estimation  2.0012137656279188
#variance naive  0.007272659524214782
#variance AV  0.003436475718600683


## **Use case 1:  Option Pricing in Finance**

price of a call option in the Black-Scholes model with maturity T and strike K.

$$
\psi(X) = e^{-rT} \left(\lambda \, e^{\sigma \sqrt{T}\, X} - K \right)_+
$$

where

$$
\lambda = S_0 \, e^{\bigl(r - \tfrac{\sigma^2}{2}\bigr) T}.
$$



$$
X_1, X_2, \dots, X_n \sim \text{N}(0,1) \quad \text{i.i.d.}
$$

$$
\mathbb{E}\bigl(\psi(X)\bigr).
$$


In [ ]:
def _f(S0,X,r,T,sigma,K):
    lamda=S0*np.exp((r-0.5*sigma**2)*T)
    return np.exp(-r*T)*np.max(lamda*np.exp(sigma*np.sqrt(T)*X)-K,0)
f = np.vectorize(_f, otypes=[float])#or use np.maximum()

# Parameters
S0 = 100   # Initial stock price
K = 100    # Strike price
r = 0.05   # Risk-free rate
sigma = 0.2 # Volatility
T = 1      # Maturity (1 year)

N=10**3
U = np.random.uniform(0, 1, N)
V = np.random.uniform(0, 1, N)

# Apply Box-Muller Transform
X = np.sqrt(-2 * np.log(U)) * np.cos(2 * np.pi * V)
X_prime=-X

estimator=np.mean(f(S0,X,r,T,sigma,K))
var_naive=np.var(f(S0,X,r,T,sigma,K))/N
var_AV=np.var(f(S0,X,r,T,sigma,K)+f(S0,X_prime,r,T,sigma,K))/(4*N)
print("#expectation estimation ",estimator)
print("#variance naive ",var_naive)
print("#variance AV ",var_AV)
 

# 95% Confidence Interval
lower_bound_naive,upper_bound_naive = estimator - 1.96 * np.sqrt(var_naive),estimator + 1.96 * np.sqrt(var_naive)
lower_bound_AV,upper_bound_AV = estimator - 1.96 * np.sqrt(var_AV),estimator + 1.96 * np.sqrt(var_AV)

print("95% Confidence Interval NAIVE:", lower_bound_naive, upper_bound_naive)
print("95% Confidence Interval AV:", lower_bound_AV, upper_bound_AV)

#expectation estimation  4.910072211501452
#variance naive  0.41053048722776797
#variance AV  0.007866796389109153
95% Confidence Interval NAIVE: 3.6542482111995813 6.165896211803323
95% Confidence Interval AV: 4.736230084822095 5.08391433818081
